# Detecão de Anomalias - Autoencoder

## Importações

In [2]:
## autoencoder
from keras import models
from keras import layers
from keras import metrics

## utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Preparação dos dados

In [3]:
## lendo os dados e dropando a coluna id
df = (
    pd.read_csv('data/porto_seguro_reduced.csv')
    .drop('id',axis=1)
)

## guardando as respostas
target = df.target

## dropando a coluna target
df.drop('target',axis=1,inplace=True)

## inspecionando o dataset
df.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,5,1,4,1,6,0,0,1,0,0,...,4,2,4,9,0,1,0,0,0,0
1,5,1,5,1,0,0,0,0,1,0,...,4,0,3,10,0,0,1,0,0,0
2,0,1,10,0,0,1,0,0,0,0,...,4,3,2,7,0,1,0,0,0,0
3,1,2,2,1,0,1,0,0,0,0,...,5,0,4,8,0,1,0,1,0,0
4,0,1,5,0,0,0,0,1,0,0,...,7,0,5,4,0,0,0,0,1,0


In [4]:
## verificando os outliers

target.value_counts(normalize=True)

0    0.96481
1    0.03519
Name: target, dtype: float64

In [5]:
## separando em treino e teste

x_train, x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((80000, 57), (20000, 57))

In [6]:
## preprocessendo as features

mx = MinMaxScaler()
x_train_scaled = mx.fit_transform(x_train)
x_test_scaled = mx.transform(x_test)

## Construção do Autoencoder

In [7]:
class AERemover():
    def __init__(self,treino,teste):
        self.treino = treino
        self.teste = teste
        self._build()
        
    def _build(self):
        
        self.encoder = models.Sequential()
        self.encoder.add(layers.Dense(32))

In [8]:
AERemover(x_train_scaled,x_test_scaled)

2021-10-15 18:56:10.991297: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-15 18:56:10.992379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 980 Ti computeCapability: 5.2
coreClock: 1.228GHz coreCount: 22 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 313.37GiB/s
2021-10-15 18:56:10.992426: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-15 18:56:10.992492: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-10-15 18:56:10.992538: I tensorflow/stream_executor/platform/